In [18]:
## GET Earnings List
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

def get_earnings_dates(symbol):
    r = requests.get('https://finance.yahoo.com/calendar/earnings?symbol={}'.format(symbol))
    soup = bs(r.content, 'lxml')
    # Using attribute = value selector
    earnigs_df = pd.DataFrame() 
    earnigs_df['dates'] = [td.text for td in soup.select('[aria-label="Earnings Date"]')]
    earnigs_df['estimated'] = [st.text for st in soup.select('[aria-label="EPS Estimate"]')]
    earnigs_df['reported'] = [rp.text for rp in soup.select('[aria-label="Reported EPS"]')]
    earnigs_df['surprise'] = [sp.text for sp in soup.select('[aria-label="Surprise(%)"]')]
    return earnigs_df

earnings_lst = get_earnings_dates('AAPL')
earnings_lst

,dates,estimated,reported,surprise
0,"Oct 27, 2021, 5 AMEST",-,-,-
1,"Jul 28, 2021, 5 AMEST",-,-,-
2,"Apr 28, 2021, 5 AMEST",-,-,-
3,"Jan 27, 2021, 4 PMEST",1.4,-,-
4,"Oct 29, 2020, 12 AMEST",0.7,0.73,+4.29
...,...,...,...,...
95,"Jul 17, 1996, 12 AMEST",-0.04,-0.02,+51.62
96,"Apr 17, 1996, 12 AMEST",-0.05,-0.07,-40.6
97,"Jan 17, 1996, 12 AMEST",-0.01,-0.02,-58.6
98,"Oct 18, 1995, 12 AMEST",0.02,0.02,-8


In [19]:
## GET Next Earning
import datetime
import re

def get_now():
    d_format = '%b %d %Y'
    return datetime.datetime.strptime(datetime.datetime.today().strftime(d_format), d_format)

def parse_enring_date(single_date):
    # Clear date string
    date_time_str = single_date.replace(',', '').replace(
        ' AMEST', 'AM').replace(' PMEST', 'PM')
    # Reshape
    date_time = datetime.datetime.strptime(date_time_str, '%b %d %Y %I%p')
    return date_time

def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

def get_next_erarning(dates_lst):
    now = get_now()
    date_list = [ parse_enring_date(e_date) for e_date in dates_lst if parse_enring_date(e_date) >= now ]
    result = False if not date_list else nearest(date_list, now)
    return result

get_next_erarning(earnings_lst['dates'])

datetime.datetime(2021, 1, 27, 16, 0)

In [1]:
## Dividends
import investpy
dividends = investpy.stocks.get_stock_dividends(stock='AAPL', country='united states')
dividends

,Date,Dividend,Type,Payment Date,Yield
0,2020-11-06,0.205,quarterly,2020-11-12,"0,63%"
1,2020-08-31,0.205,quarterly,2020-08-28,"0,74%"
2,2020-08-07,0.820,quarterly,2020-08-13,"0,72%"
3,2020-05-08,0.820,quarterly,2020-05-14,"0,86%"
4,2020-02-07,0.770,quarterly,2020-02-13,"1,07%"
...,...,...,...,...,...
59,1989-11-17,0.110,trailing_twelve_months,1989-11-24,-
60,1989-08-21,0.100,trailing_twelve_months,1989-08-28,-
61,1989-05-22,0.100,trailing_twelve_months,1989-05-29,-
62,1989-02-17,0.100,trailing_twelve_months,1989-02-24,-


In [5]:
# https://pypi.org/project/yahoofinancials/
from yahoofinancials import YahooFinancials

ticker = 'AAPL'
yahoo_financials = YahooFinancials(ticker)


In [7]:
start_date = '2020-01-15'
end_date = '2021-05-15'
yahoo_financials = YahooFinancials(['AAPL'])
print(yahoo_financials.get_daily_dividend_data(start_date, end_date))

{'AAPL': [{'date': 1581085800, 'formatted_date': '2020-02-07', 'amount': 0.1925}, {'date': 1588944600, 'formatted_date': '2020-05-08', 'amount': 0.205}, {'date': 1596807000, 'formatted_date': '2020-08-07', 'amount': 0.205}, {'date': 1604673000, 'formatted_date': '2020-11-06', 'amount': 0.205}]}


In [20]:
import yfinance as yf

aapl = yf.Ticker('AAPL')

aapl.calendar

,Value
Earnings Date,2021-01-27 00:00:00
Earnings Average,1.4
Earnings Low,1.23
Earnings High,1.57
Revenue Average,102458000000
Revenue Low,97739000000
Revenue High,110211000000


In [22]:
from datetime import datetime
# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(aapl.info['exDividendDate']).strftime('%Y-%m-%d %H:%M:%S'))


2020-11-06 00:00:00


In [23]:
print(datetime.utcfromtimestamp(aapl.info['lastDividendDate']).strftime('%Y-%m-%d %H:%M:%S'))


2020-11-06 00:00:00


In [24]:
aapl.dividends

Date
1987-05-11    0.00054
1987-08-10    0.00054
1987-11-17    0.00071
1988-02-12    0.00071
1988-05-16    0.00071
               ...   
2019-11-07    0.19250
2020-02-07    0.19250
2020-05-08    0.20500
2020-08-07    0.20500
2020-11-06    0.20500
Name: Dividends, Length: 69, dtype: float64

In [25]:
# ========== FED MEETINGS
from FedTools import MonetaryPolicyCommittee
from FedTools import BeigeBooks
from FedTools import FederalReserveMins

fed_mins = FederalReserveMins(
            main_url = 'https://www.federalreserve.gov', 
            calendar_url ='https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm',
            historical_split = 2014,
            verbose = True,
            thread_num = 10)

dataset = fed_mins.find_minutes()

Constructing Links.
Extracting Federal Reserve Minutes.
Retrieving articles.
...............................................................................................................................................................................................................................

In [26]:
fed_mins

In [27]:
dataset

,Federal_Reserve_Mins
1993-02-03,A meeting of the Federal Open Market Committee...
1993-03-23,A meeting of the Federal Open Market Committee...
1993-05-18,A meeting of the Federal Open Market Committee...
1993-07-07,A meeting of the Federal Open Market Committee...
1993-08-17,A meeting of the Federal Open Market Committee...
...,...
2020-06-10,"The Federal Reserve, the central bank of the U..."
2020-07-29,"The Federal Reserve, the central bank of the U..."
2020-09-16,"The Federal Reserve, the central bank of the U..."
2020-11-05,"The Federal Reserve, the central bank of the U..."
